In [98]:
import pandas as pd
import numpy as np
import copy
import itertools
# Package for Levenstein string distance
import jellyfish
import time

In [34]:
path = "C://Users//Dimitri//Desktop//ENSAE3A//NetworkData//Data//"

In [35]:
attrs = pd.read_csv(path + "attrs_sub.csv")

In [50]:
search = database["author"].str.contains("Smidt")
search2 = database["title"].str.contains("Policy")

In [51]:
ind = search[search == True].index
ind2 = search2[search2 == True].index
inter = ind.intersection(ind2).shape

In [36]:
def authors_parser(authors_string, sep=";"):
    return authors_string.split("; ")

In [104]:
def normalized_edit_distance(str1, str2):
    dist = jellyfish.levenshtein_distance(str1, str2)
    return dist / max(len(str1), len(str2))

In [109]:
dd = "Noel D."
d = "Ramon E."
normalized_edit_distance(dd, d)

0.75

In [37]:
attrs["authors_list"] = attrs["authors"].apply(authors_parser)

In [38]:
attrs

,url,title,authors,date,jel_code,keywords,editor,journal,article_id,authors_list
0,https://ideas.repec.org/a/oup/qjecon/v1y1886i1...,The Reaction in Political Economy,Charles F. Dunbar,1886-02-02,NaN,NaN,oup,qjecon,v1y1886i1p1-27..html,[Charles F. Dunbar]
1,https://ideas.repec.org/a/oup/qjecon/v1y1886i1...,Private Monopolies and Public Rights,Arthur T. Hadley,1886-02-02,NaN,NaN,oup,qjecon,v1y1886i1p28-44..html,[Arthur T. Hadley]
2,https://ideas.repec.org/a/oup/qjecon/v1y1886i1...,Silver Before Congress in 1886,S. Dana Horton,1886-02-02,NaN,NaN,oup,qjecon,v1y1886i1p45-75..html,[S. Dana Horton]
3,https://ideas.repec.org/a/oup/qjecon/v1y1886i1...,"The Arithmetic, Geometric, and Harmonic Means",F. Coggeshall,1886-02-02,NaN,NaN,oup,qjecon,v1y1886i1p83-86..html,[F. Coggeshall]
4,https://ideas.repec.org/a/oup/qjecon/v1y1886i1...,Legislation for Labor Arbitration,H. M. Williams,1886-02-02,NaN,NaN,oup,qjecon,v1y1886i1p86-91..html,[H. M. Williams]
5,https://ideas.repec.org/a/oup/qjecon/v1y1886i1...,Correspondence,Arthur Mangin,1886-02-02,NaN,NaN,oup,qjecon,v1y1886i1p91-102..html,[Arthur Mangin]
6,https://ideas.repec.org/a/oup/qjecon/v1y1887i2...,An Historical Sketch of the Knights of Labor,Carroll D. Wright,1887-02-02,NaN,NaN,oup,qjecon,v1y1887i2p137-168..html,[Carroll D. Wright]
7,https://ideas.repec.org/a/oup/qjecon/v1y1887i2...,The Disposition of Our Public Lands,Albert Bushnell Hart,1887-02-02,NaN,NaN,oup,qjecon,v1y1887i2p169-183..html,[Albert Bushnell Hart]
8,https://ideas.repec.org/a/oup/qjecon/v1y1887i2...,The South-Western Strike of 1886,F. W. Taussig,1887-02-02,NaN,NaN,oup,qjecon,v1y1887i2p184-222..html,[F. W. Taussig]
9,https://ideas.repec.org/a/oup/qjecon/v1y1887i2...,Marshall's Theory of Value and Distribution,J. Laurence Laughlin,1887-02-02,NaN,NaN,oup,qjecon,v1y1887i2p227-232..html,[J. Laurence Laughlin]


In [115]:
def parse_names(author):
    if "," in author:
        splitted = author.split(", ")
        lastname = splitted[0]
        firstname = splitted[1]
        if firstname[0] == " ":
            firstname = firstname[1:]
    else:
        splitted = author.split(" ")
        lastname = splitted[-1]
        if len(splitted) > 1:
            firstname = ""
            for part in splitted[: -1]:
                firstname += " " + part
        else:
            firstname = ""
    return firstname, lastname

In [111]:
def sortby_firstname_len(author):
    firstname, lastname = parse_names(author)
    return len(firstname)
   

In [113]:
# Fill all authors
authors = []
for author in attrs["authors_list"]:
    authors += author
# Remove duplicates
authors = list(set(authors))
authors_list = authors[0:1000]

In [118]:
# Sortby first name length
start = time.clock()
authors_list.sort(key=sortby_firstname_len, reverse=True)
end = time.clock()
print(end - start)

0.002390161207586061


In [119]:
authors_list

['Population Dynamics: A New Economic Approach',
 'Practice: Lionel Robbins Lectures',
 'Nils-Henrik M von der Fehr',
 'Chen, Zhuoqiong (Charlie)',
 'Elizabeth Waterman Gilboy',
 'Katariina Nilsson Hakkala',
 'Terra, Maria Cristina T.',
 'Katherine Bement Davis',
 'Frederick Emory Haynes',
 'William Wallace Hewett',
 'Stephanie Riegg Cellini',
 'Sushka, Marie Elizabeth',
 'Antoine Faure- Grimaud',
 'Daniel Valente Dantas',
 'Tang, Dragon Yongjun',
 'Wallack, Jessica Seddon',
 'Weiss Jeffrey H.',
 'Ellis, Christopher J.',
 ' Bailey, Warren Bernard',
 'Wouter J. Den Haan',
 'Broek, Julien van den',
 'Kelton, Christina M L',
 'Christina H. Paxson',
 'Nicolas P.B. Bollen',
 'James Parker Hall',
 'Giampiero M. Gallo',
 'Bruce Truitt Elmslie',
 'Jean Jacques Laffont',
 'Ingrid M. T. Rohde',
 'Wang, Steven Shuye',
 'Williams, Arlington W.',
 'Henry Allen Latane',
 'Priyodorshi Banerjee',
 'Ardeshir J. Dalal',
 'Juan Carlos Conesa',
 'Penelope C. Hartland',
 'Jost, Peter-Jurgen',
 'Alastair R.

In [139]:
start = time.clock()
thresh = 0.25
authors_frame = pd.DataFrame(columns=["first_name", "last_name", "similar"])
for author in authors_list:
    firstname, lastname = parse_names(author)
    search = authors_frame["last_name"].str.contains(lastname)
    ind_search = search[search == True].index
    exists_similar = False
    for ind in ind_search :
        dist = normalized_edit_distance(authors_frame["first_name"][ind], 
                                        firstname)
        dist_test = dist <= thresh
        if dist_test :
            exists_similar = True
            authors_frame["similar"][ind].append(author)    
        first_letter_test1 = (len(firstname) == 1) and (firstname[0] == authors_frame["first_name"][ind][0])
        if first_letter_test1:
            exists_similar = True
            authors_frame["similar"][ind].append(author)     
    if not exists_similar:
        df = pd.DataFrame(columns=["first_name", "last_name", "similar"], index=[0])
        df.set_value(0, "first_name", firstname)
        df.set_value(0, "last_name", lastname)
        df.set_value(0, "similar", [author])
        authors_frame = authors_frame.append(df, ignore_index=True)
end = time.clock()
print(end - start)

4.069191082890029


In [144]:
inds = []
for b in authors_frame.iterrows():
    if len(b[1]["similar"]) > 1:
        inds.append(b[0])

In [152]:
authors_frame

,first_name,last_name,similar
0,Population Dynamics: A New Economic,Approach,[Population Dynamics: A New Economic Approach]
1,Practice: Lionel Robbins,Lectures,[Practice: Lionel Robbins Lectures]
2,Nils-Henrik M von der,Fehr,[Nils-Henrik M von der Fehr]
3,Zhuoqiong (Charlie),Chen,"[Chen, Zhuoqiong (Charlie)]"
4,Elizabeth Waterman,Gilboy,[Elizabeth Waterman Gilboy]
5,Katariina Nilsson,Hakkala,[Katariina Nilsson Hakkala]
6,Maria Cristina T.,Terra,"[Terra, Maria Cristina T.]"
7,Katherine Bement,Davis,[Katherine Bement Davis]
8,Frederick Emory,Haynes,[Frederick Emory Haynes]
9,William Wallace,Hewett,[William Wallace Hewett]


In [151]:
authors_frame.iloc[741]

first_name                                M. G.
last_name                              Marchand
similar       [M. G. Marchand, Marchand, M. G.]
Name: 741, dtype: object

In [45]:
authors

['Jones, Lawrence D',
 'Danyang Xie',
 'Feng Wang',
 'Amelia Santos-Paulino',
 'Mark Maier',
 'Johnson, William C.',
 'Ramon E. Johnson',
 'Stephen G. Cecchetti',
 'W. D. Conklin',
 'Liu, Xiaodong',
 'David Canning',
 ' Smidt, S',
 'Kelton, Christina M L',
 'Porter, Michael G',
 'Yuan, Haishan',
 'Detemple Jerome',
 'Wichardt, Philipp C.',
 'Chen, Zhuoqiong (Charlie)',
 'Chen, Yifan',
 'Ezra Solomon',
 'Khandker, Rezaul K',
 'Tobias Cwik',
 'Pettersen, Oystein',
 'Paolo F. Volpin',
 'Ferge, Zsuzsa',
 'Gordy, Michael B.',
 'Alexander Klein',
 'Xavier Vives',
 'Tasci, Murat',
 'Elizabeth Caucutt',
 'Rebitzer, James B',
 'Sampson, Roy J',
 'Paul Willen',
 'K. Newey, Whitney',
 'Bucă, Andra',
 'Baik, Kyung Hwan',
 'Ross, Howard N.',
 'Wang, Steven Shuye',
 'Segura, Anatoli',
 'Andrew Leicester',
 'Ron Holzman',
 'Krautkraemer, Jeffrey A.',
 'Montalvo, Jose G.',
 'Xiaoji Lin',
 'Dias, D.A.',
 'Simon, Leo',
 'Unal, Haluk',
 'Zweifel, Peter',
 'Adrien Vigier',
 'Jason Hwang',
 'Mary Norris',


In [46]:
search = attrs["authors"].str.contains("McCulloch")
inds = search[search == True].index
inds

Int64Index([ 3233,  8748,  8930, 11844, 12634, 13196, 13535, 15586, 16708,
            22473, 22533, 24696, 24861, 27642, 33168, 35140, 37362, 39752,
            39821, 39903, 40049, 43542, 43807, 44346, 44353, 45200, 45878,
            46793, 48995, 52798, 52888, 53358, 60522, 68443],
           dtype='int64')

In [47]:
attrs.iloc[inds]

,url,title,authors,date,jel_code,keywords,editor,journal,article_id,authors_list
3233,https://ideas.repec.org/a/oup/qjecon/v89y1975i...,Operational Aspects of the Siegel Paradox,J. Huston McCulloch,1975-02-02,NaN,NaN,oup,qjecon,v89y1975i1p170-172..html,[J. Huston McCulloch]
8748,https://ideas.repec.org/a/ucp/jpolec/v83y1975i...,An Estimate of the Liquidity Premium,"McCulloch, J Huston",1975-02-02,NaN,NaN,ucp,jpolec,v83y1975i1p95-119.html,"[McCulloch, J Huston]"
8930,https://ideas.repec.org/a/ucp/jpolec/v85y1977i...,"Factor Mobility, Regional Development, and the...","McCulloch, Rachel; Yellen, Janet L",1977-02-02,NaN,NaN,ucp,jpolec,v85y1977i1p79-96.html,"[McCulloch, Rachel, Yellen, Janet L]"
11844,https://ideas.repec.org/a/aea/aecrev/v63y1973i...,A Note on Proportionally Distributed Quotas,"McCulloch, Rachel; Johnson, Harry G",1973-02-02,NaN,NaN,aea,aecrev,v63y1973i4p726-32.html,"[McCulloch, Rachel, Johnson, Harry G]"
12634,https://ideas.repec.org/a/aea/aecrev/v67y1977i...,Trade as Aid: The Political Economy of Tariff ...,"McCulloch, Rachel; Pinera, Jose",1977-02-02,NaN,NaN,aea,aecrev,v67y1977i5p959-67.html,"[McCulloch, Rachel, Pinera, Jose]"
13196,https://ideas.repec.org/a/aea/aecrev/v70y1980i...,"The Ban on Indexed Bonds, 1933-77","McCulloch, J Huston",1980-02-02,NaN,NaN,aea,aecrev,v70y1980i5p1018-21.html,"[McCulloch, J Huston]"
13535,https://ideas.repec.org/a/aea/aecrev/v72y1982i...,Incentives and Proxies for Indexed Bond Issues...,"McCulloch, J Huston",1982-02-02,NaN,NaN,aea,aecrev,v72y1982i3p566-68.html,"[McCulloch, J Huston]"
15586,https://ideas.repec.org/a/aea/aecrev/v83y1993i...,The Optimality of Free Trade: Science or Relig...,"McCulloch, Rachel",1993-02-02,NaN,NaN,aea,aecrev,v83y1993i2p367-71.html,"[McCulloch, Rachel]"
16708,https://ideas.repec.org/a/aea/aecrev/v90y2000i...,"Floating, Fixed, or Super-Fixed? Dollarization...",Rachel McCulloch; Blake LeBaron,2000-02-02,F33; J71; J15; F31; O15; I12; O19; D31; J13; P33,NaN,aea,aecrev,v90y2000i2p32-37.html,"[Rachel McCulloch, Blake LeBaron]"
22473,https://ideas.repec.org/a/ecm/emetrp/v52y1984i...,A Hazard Rate Approach to the Timing of Births,"Newman, John L; McCulloch, Charles E",1984-02-02,NaN,NaN,ecm,emetrp,v52y1984i4p939-61.html,"[Newman, John L, McCulloch, Charles E]"


In [67]:
it_list = copy.copy(authors)
authors_tuple = itertools.product(authors, authors)
collaborations = pd.DataFrame(columns=["author1", "author2", "n_collaborations"])
for author in it_list:
    collaboration[]
    
    

Traceback (most recent call last):
  File "C:\Users\Dimitri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2885, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-72-05571270874a>", line 1, in <module>
    list(authors_tuple)
MemoryError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Dimitri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2888, in run_code
    sys.excepthook = old_excepthook
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Dimitri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 1827, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent 

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



TypeError: Can't convert 'list' object to str implicitly